<a href="https://colab.research.google.com/github/Athugodage/RuLawSimplification/blob/main/two_way/interference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.0 MB/s 
     |████████████████████████████████| 163 kB 69.6 MB/s 
     |████████████████████████████████| 6.6 MB 35.4 MB/s 


In [4]:
!pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import razdel
import json

In [6]:
!git clone https://github.com/Athugodage/RuLawSimplification.git

Cloning into 'RuLawSimplification'...
remote: Enumerating objects: 176, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 176 (delta 47), reused 29 (delta 8), pack-reused 76
Receiving objects: 100% (176/176), 44.10 MiB | 35.56 MiB/s, done.
Resolving deltas: 100% (77/77), done.


In [8]:
!unzip /content/RuLawSimplification/Important_docs/important_docs.zip

Archive:  /content/RuLawSimplification/Important_docs/important_docs.zip
  inflating: important_docs2.csv     
  inflating: important_docs1.csv     


In [9]:
with open('/content/RuLawSimplification/two_way/two_way.json', 'r', encoding='utf8') as rp:
    a = json.load(rp)

In [10]:
len(a)

201

Теперь, после проверки, можно начать

In [11]:
data = pd.read_csv('important_docs1.csv', delimiter=';', encoding='windows-1251')
data

,Название документа,Ссылка,Текст,Комментарий РГ,Дата
0,Постановление Конституционного Суда Российской...,https://rg.ru/2022/02/10/postanovlenie-dok.html,Именем Российской Федерации Конституционный С...,В том числе компания поставляет тепло и горячу...,10 февраля 2022 г.
1,Постановление Конституционного Суда Российской...,https://rg.ru/2022/02/03/postanovlenie-dok.html,Именем Российской Федерации Конституционный С...,Соответствующий документ опубликован на 10-й с...,3 февраля 2022 г.
2,Постановление Конституционного Суда Российской...,https://rg.ru/2022/01/25/ks-dok.html,Именем Российской Федерации Конституционный С...,В 2015 году Татьяна Малкова из Челябинской обл...,25 января 2022 г.
3,Приказ Министерства экономического развития Ро...,https://rg.ru/2022/01/25/intellekt-dok.html,"В целях реализации федерального проекта ""Искус...",Регулярный мониторинг позволит получать объект...,25 января 2022 г.
4,Приказ Министерства экономического развития Ро...,https://rg.ru/2022/01/25/rabota-dok.html,"В целях реализации федерального проекта ""Искус...",Регулярный мониторинг позволит получать объект...,25 января 2022 г.
...,...,...,...,...,...
851,"Федеральный закон от 23 мая 2016 г. N 147-ФЗ ""...",https://rg.ru/2016/05/25/ekipaj-dok.html,Принят Государственной Думой 10 мая 2016 года ...,"Закон, который подписал президент Владимир Пут...",25 мая 2016 г.
852,"Федеральный закон от 23 мая 2016 г. N 143-ФЗ ""...",https://rg.ru/2016/05/25/pensii-kategorii-dok....,Принят Государственной Думой 11 мая 2016 года ...,Закон об этом подписан президентом страны. Вст...,25 мая 2016 г.
853,Постановление Пленума Верховного Суда Российск...,https://rg.ru/2016/05/18/vstamozh-dok.html,В целях обеспечения единства практики применен...,В спорные ситуации на таможне попадают не толь...,18 мая 2016 г.
854,"Федеральный закон от 3 мая 2016 г. N 140-ФЗ ""О...",https://rg.ru/2016/05/06/duma-dok.html,Принят Государственной Думой 22 апреля 2016 го...,Хотя и раньше депутатам приходилось исключать ...,6 мая 2016 г.


In [12]:

text = [full_text for full_text in data['Текст']]
comment = [full_com for full_com in data['Комментарий РГ']]

In [13]:
model_checkpoint = 'cointegrated/rubert-base-cased-nli-twoway'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()


Downloading:   0%|          | 0.00/545 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.62M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/712M [00:00<?, ?B/s]

In [14]:
def exchange(com, tekst):   
   
    with torch.inference_mode():
        out = model(**tokenizer(com, tekst, padding=True, truncation=True,max_length=50, add_special_tokens = True, return_tensors='pt').to(model.device))
        proba = torch.softmax(out.logits, -1).cpu().numpy()[0]
        x1 = {v: proba[k] for k, v in model.config.id2label.items()}
    with torch.inference_mode():
        out = model(**tokenizer(tekst, com, padding=True, truncation=True,max_length=50, add_special_tokens = True, return_tensors='pt').to(model.device))
        proba = torch.softmax(out.logits, -1).cpu().numpy()[0]
        x2 = {v: proba[k] for k, v in model.config.id2label.items()}

    x = x1['entailment'] + x2['entailment']
    return x

In [ ]:
from tqdm import tqdm

json_dic = []

# 0-100 ready
for n in tqdm(range(301, 501)):
    #n = 0
    article = text[n]
    com_article =  comment[n]
    # full documents and comments
    first_dic = {'article number': n, 
                 'article name': data['Название документа'][n],
                 'article link': data['Ссылка'][n],
                 'article text': []}
    json_dic.append(first_dic)

    text_sent = [_.text for _ in razdel.sentenize(article)]
    comment_sent = [_.text for _ in razdel.sentenize(com_article)]

    # full documents and comments divided on sentences - borders are marked
    #medium_list = []
    for com in comment_sent:
        # sentence of a comment

        small_dic = {}
        for tekst in text_sent:
            # sentence of a text

            a = exchange(com, tekst)
            small_dic[tekst] = a

        maximum = max(small_dic, key=small_dic.get)
        second_dic = {'entailment score': float(small_dic[maximum]), 
                      'comment': str(com), 
                      'text': str(maximum)}
        json_dic[-1]['article text'].append(second_dic)
  

 68%|██████▊   | 137/200 [1:07:00<41:13, 39.26s/it]

In [21]:
json_dic[9]

{'article number': 210,
 'article name': 'Федеральный закон от 8 декабря 2020 г. N 407-ФЗ "О внесении изменений в Трудовой кодекс Российской Федерации в части регулирования дистанционной (удаленной) работы и временного перевода работника на дистанционную (удаленную) работу по инициативе работодателя в исключительных случаях" — Российская газета',
 'article link': 'https://rg.ru/2020/12/11/udalenka-dok.html',
 'article text': [{'entailment score': 1.1914845705032349,
   'comment': 'Он закрепляет три формы удаленной работы.',
   'text': 'Порядок предоставления дистанционному работнику, выполняющему дистанционную работу на постоянной основе в соответствии с трудовым договором или дополнительным соглашением к трудовому договору, ежегодного оплачиваемого отпуска и иных видов отпусков определяется коллективным договором, локальным нормативным актом, принятым с учетом мнения выборного органа первичной профсоюзной организации, трудовым договором в соответствии с настоящим Кодексом и иными акта

In [24]:
n = 0
num = 0
best_pairs = {}

for article in json_dic:
    for pair in article['article text']:
        num += 1
        if pair['entailment score'] > 0.75:
            n += 1
            

print(n, '/', num)

245 / 1686


In [36]:
!wget https://github.com/Athugodage/RuLawSimplification/blob/630a669f0d8726a68af8e521d1d1da3fce3081db/two_way/two_way.json

--2022-10-07 22:45:50--  https://github.com/Athugodage/RuLawSimplification/blob/630a669f0d8726a68af8e521d1d1da3fce3081db/two_way/two_way.json
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘two_way.json’

two_way.json            [ <=>                ] 135.49K  --.-KB/s    in 0.07s   

2022-10-07 22:45:50 (1.84 MB/s) - ‘two_way.json’ saved [138744]



In [44]:
with open('two_way.json', 'a', encoding='utf8') as fp:
    dic = json.dump(json_dic, fp, ensure_ascii=False, 
                    check_circular=True, allow_nan=True, cls=None, indent=3, 
                    separators=None, default=None, sort_keys=False)

In [48]:
with open('nli-300.json', 'r', encoding='utf8') as rp:
    a = json.load(rp)

In [55]:
a[102]

{'article number': 102,
 'article name': 'Федеральный закон от 30 апреля 2021 г. N 135-ФЗ "О внесении изменений в статью 28 Федерального закона "О ветеранах" — Российская газета',
 'article link': 'https://rg.ru/2021/05/06/veterany-dok.html',
 'article text': [{'entailment score': 0.10261330753564835,
   'comment': 'Если человек погиб на войне, уже обладая таким статусом, вопросов с передачей его родным прав на дополнительные соцгарантии не возникает.',
   'text': 'Реализация";'},
  {'entailment score': 0.057955726981163025,
   'comment': 'Но если участие военного в боях при его жизни не было подтверждено ветеранской "корочкой", возникала проблема.',
   'text': 'Выдача удостоверения члена семьи ветерана боевых действий осуществляется членам семей лиц, указанных в пункте 1 статьи 3 настоящего Федерального закона, погибших (умерших) и не получивших при жизни удостоверение ветерана боевых действий, на основании документов, подтверждающих факт гибели (смерти) указанных лиц, а также факт их